### Streming Twitter

In [1]:
# pacotes de autenticacao
#!pip install requests_oauthlib
#!pip install twython
#!pip install nltk==3.5

In [2]:
# Importando modulos
from pyspark.streaming import StreamingContext
from pyspark import SparkContext
from requests_oauthlib import oauth1_session
from operator import add
import requests_oauthlib
from time import gmtime, strftime
import requests
import time
import string
import ast
import json

In [3]:
help("requests_oauthlib")

Help on package requests_oauthlib:

NAME
    requests_oauthlib

PACKAGE CONTENTS
    compliance_fixes (package)
    oauth1_auth
    oauth1_session
    oauth2_auth
    oauth2_session

VERSION
    1.3.0

FILE
    /home/ac/anaconda3/lib/python3.7/site-packages/requests_oauthlib/__init__.py




In [4]:
# nltk
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *


In [5]:
# Frequencia de updates
INTERVAL_BATCH = 5

In [6]:
# criando o StreamingContext
ssc = StreamingContext(sc, INTERVAL_BATCH)

### Treinando o classificador
* 1 sentimento positivo
* 0 sentimento negativo

In [7]:
import os.path

In [8]:
filename = "data/dataset_analise_sentimento.csv"
if os.path.isfile(filename):
    ds = sc.textFile(filename)
else:
    print("arquivo inexistente")

In [9]:
type(ds)

pyspark.rdd.RDD

In [10]:
ds.take(5)

['ItemID,Sentiment,SentimentSource,SentimentText',
 '1,0,Sentiment140,                     is so sad for my APL friend.............',
 '2,0,Sentiment140,                   I missed the New Moon trailer...',
 '3,1,Sentiment140,              omg its already 7:30 :O',
 "4,0,Sentiment140,          .. Omgaga. Im sooo  im gunna CRy. I've been at this dentist since 11.. I was suposed 2 just get a crown put on (30mins)..."]

In [11]:
header = ds.first()
dataset = ds.filter(lambda line : line != header)

In [12]:
type(dataset)

pyspark.rdd.PipelinedRDD

In [13]:
# Essa função separa as colunas em cada linha, cria uma tupla e remove a pontuação.
def get_row(line):
    row = line.split(",")
    sentimento = row[1]
    tweet = row[3].strip()
    translator = str.maketrans({key: None for key in string.punctuation})
    tweet = tweet.translate(translator)
    tweet = tweet.split(" ")
    tweet_lower = []
    for word in tweet:
        tweet_lower.append(word.lower())
    return (tweet_lower, sentimento)

In [14]:
dataset_treino = dataset.map(lambda line: get_row(line))

In [15]:
sentiment_analyzer = SentimentAnalyzer()

In [16]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/ac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from IPython.display import Image
Image(url = "images/ntlkdata.png")


In [18]:
stopwords_all = []
for word in stopwords.words("english"):
    stopwords_all.append(word)
    stopwords_all.append(word + "_NEG")

In [19]:
dataset_treino_amostra = dataset_treino.take(1000)

In [20]:
all_words_neg = sentiment_analyzer.all_words([mark_negation(doc) for doc in dataset_treino_amostra])
all_words_neg_nostops = [x for x in all_words_neg if x not in stopwords_all]

In [21]:
unigram_feats = sentiment_analyzer.unigram_word_feats(all_words_neg_nostops, top_n = 200)
sentiment_analyzer.add_feat_extractor(extract_unigram_feats, unigrams = unigram_feats)
training_set = sentiment_analyzer.apply_features(dataset_treino_amostra)

In [22]:
type(training_set)

nltk.collections.LazyMap

In [23]:
trainer = NaiveBayesClassifier.train
classifier = sentiment_analyzer.train(trainer, training_set)

Training classifier


In [24]:
test_sentence1 = [(["this", "program", "is", "bad"], "")]
test_sentence2 = [(["tough", "day", "at", "work", "today"], "")]
test_sentence3 = [(["good", "wonderful", "amazing", "awesome"], "")]

test_set1 = sentiment_analyzer.apply_features(test_sentence1)
test_set2 = sentiment_analyzer.apply_features(test_sentence2)
test_set3 = sentiment_analyzer.apply_features(test_sentence3)


In [25]:
consumer_key = "J1sHChMSqrr5as1jtmP9Duo2bV"
consumer_secret = "cg25DJ7thm1pgnWW1VrBVRmDQtmwefNeQOJxxf1Z0GSgg4tiPPK"
access_key = "25630007515-sIbKfRbuAbIftp71x0pc0cXgt5uUWZgPTaqvswu"
acesss_token_secret = "VIpm4tHmO9zfUGFDaK6Oz19Kjeqw9UyAmdgkEd6umxETFN"


In [26]:
search_term = "Lamar"
sample_url = "https://stream.twitter.com/1.1/statuses/samples.json"
filter_url = "https://stream.twitter.com/1.1/statuses/filter.json?track="+search_term

In [27]:
auth = requests_oauthlib.OAuth1(consumer_key, consumer_secret, access_key, acesss_token_secret)

In [28]:
rdd = ssc.sparkContext.parallelize([0])
stream = ssc.queueStream([], default = rdd)

In [29]:
type(stream)

pyspark.streaming.dstream.DStream

In [30]:
NUM_TWEETS = 500

In [31]:
def tfunc(t, rdd):
    return rdd.flatMap(lambda x: stream_twitter_data())

def stream_twitter_data():
    response = requests.get(filter_url, auth = auth, stream = True)
    print(filter_url, response)
    count = 0
    for line in response.iter_lines():
        try:
            if count > NUM_TWEETS:
                break
            post  = json.loads(line.decode("utf-8"))
            contents = [post["text"]]
            count += 1
            yield str(contents)
        except:
            result = False
            

In [32]:
stream = stream.transform(tfunc)

In [33]:
coord_stream = stream.map(lambda line: ast.literal_eval(line))

In [34]:
def classifica_tweet(tweet):
    sentece = [(tweet, "")]
    test_set = sentiment_analyzer.apply_features(sentence)
    print(tweet, classifier.classify(test_set[0][0]))
    return (tweet, classifier.classify(test_set[0][0]))

In [35]:
def get_tweet_text(rdd):
    for line in rdd:
        tweet = line.strip()
        translator = str.maketrans({key: None for key in string.punctuation})
        tweet = tweet.translate(translator)
        tweet = tweet.split(" ")
        tweet_lower = []
        for word in tweet:
            tweet_lower.append(word.lower())
        return (classifica_tweet(tweet_lower))

In [36]:
resultados = []

In [37]:
def output_rdd(rdd):
    global resultados
    pairs = rdd.map(lambda x: (get_tweet_text(x)[1], 1))
    counts = pairs.reduceByKey(add)
    output = []
    for count in counts.collect():
        output.append(count)
    result = [time.strftime("%I:%M:%S"), output]
    resultados.append(result)
    print(result)

In [38]:
coord_stream.foreachRDD(lambda t, rdd: output_rdd(rdd))

In [39]:
ssc.start()
# ssc.awaitTermination()

In [ ]:
cont = True
while cont:
    if len(resultados) > 5:
        cont = False

rdd_save = "data/result";time.strftime("%I%M%S")
resuldados_rdd = sc.parallelize(resultados)
resuldados_rdd.saveTextFile(rdd_save)

resuldados_rdd.collect()

In [ ]:
# ssc.stop()

### Charts